In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from datetime import datetime, timedelta
from math import radians, sin, cos, asin, sqrt
import os
from collections import defaultdict

# Geographic distance calculation function (unit: kilometers)
def geodistance(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [float(lon1), float(lat1), float(lon2), float(lat2)])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * 6371 * asin(sqrt(a))

# JSD calculation function
def js_divergence(p, q):
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)
    p /= p.sum() + 1e-9
    q /= q.sum() + 1e-9
    m = (p + q) / 2
    return 0.5 * scipy.stats.entropy(p, m) + 0.5 * scipy.stats.entropy(q, m)

# Trajectory analysis class (improved version)
class TrajectoryAnalyzer:
    def __init__(self, grid_size=0.01, lat_range=(40.5, 40.9), lon_range=(-74.25, -73.7)):
        self.grid_size = grid_size
        self.lat_min, self.lat_max = lat_range
        self.lon_min, self.lon_max = lon_range
        self.n_lat = int((self.lat_max - self.lat_min) / grid_size) + 1
        self.n_lon = int((self.lon_max - self.lon_min) / grid_size) + 1
        self.grid_total = self.n_lat * self.n_lon
        self.time_bins = 144  # 24 hours * 6 (10-minute intervals)
        self.day_of_week = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    
    def parse_real_time(self, time_str):
        """Parse time format for real data"""
        dt = datetime.strptime(time_str, '%a %b %d %H:%M:%S %z %Y')
        return dt
    
    def parse_gen_time(self, time_str):
        """Parse time format for generated data"""
        return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
    
    def time_to_bin(self, dt):
        """Convert time to time slot index"""
        total_minutes = dt.hour * 60 + dt.minute
        return min(int(total_minutes // 10), self.time_bins - 1)
    
    def latlon_to_grid(self, lat, lon):
        """Convert latitude/longitude to grid index"""
        if not (self.lat_min <= lat <= self.lat_max and self.lon_min <= lon <= self.lon_max):
            return -1  # Out of range
        
        lat_idx = int((lat - self.lat_min) / self.grid_size)
        lon_idx = int((lon - self.lon_min) / self.grid_size)
        return min(lat_idx * self.n_lon + lon_idx, self.grid_total - 1)
    
    def analyze_real_trajectories(self, real_df):
        """Analyze real trajectories (multi-day data)"""
        results = {
            'distances': [],
            'durations': [],
            'st_act_events': defaultdict(list),
            'st_loc_events': defaultdict(list),
            'daily_activity_dist': defaultdict(lambda: defaultdict(int))
        }
        
        # Group by user
        for user_id, group in real_df.groupby('userId'):
            group = group.sort_values('utcTimestamp')
            prev_dt = None
            prev_lat, prev_lon = None, None
            
            for _, row in group.iterrows():
                dt = self.parse_real_time(row['utcTimestamp'])
                day_key = dt.strftime('%Y-%m-%d')
                
                # Calculate movement distance
                if prev_dt and (dt - prev_dt) < timedelta(hours=6):  # Ignore intervals longer than 6 hours
                    dist = geodistance(prev_lon, prev_lat, row['longitude'], row['latitude'])
                    results['distances'].append(dist)
                
                # Calculate stay duration
                if prev_dt:
                    duration = (dt - prev_dt).total_seconds() / 60  # minutes
                    results['durations'].append(duration)
                
                # Create spatiotemporal events
                time_bin = self.time_to_bin(dt)
                grid_idx = self.latlon_to_grid(row['latitude'], row['longitude'])
                
                if grid_idx >= 0:
                    # Daily statistics
                    results['st_act_events'][day_key].append((time_bin, row['venueCategory']))
                    results['st_loc_events'][day_key].append((time_bin, grid_idx))
                    
                    # Daily activity distribution
                    results['daily_activity_dist'][day_key][row['venueCategory']] += 1
                
                prev_dt = dt
                prev_lat, prev_lon = row['latitude'], row['longitude']
        
        return results
    
    def analyze_generated_trajectories(self, gen_df):
        """Analyze generated trajectories (single-day simulation)"""
        results = {
            'distances': [],
            'durations': [],
            'st_act_events': [],
            'st_loc_events': [],
            'activity_dist': defaultdict(int)
        }
        
        # Group by user
        for user_id, group in gen_df.groupby('userId'):
            group = group.sort_values('timestamp')
            prev_dt = None
            
            for _, row in group.iterrows():
                dt = self.parse_gen_time(row['timestamp'])
                time_bin = self.time_to_bin(dt)
                
                # Origin event
                origin_grid = self.latlon_to_grid(row['origin_lat'], row['origin_lon'])
                if origin_grid >= 0:
                    results['st_act_events'].append((time_bin, row['origin_category']))
                    results['st_loc_events'].append((time_bin, origin_grid))
                    results['activity_dist'][row['origin_category']] += 1
                
                # Destination event
                dest_grid = self.latlon_to_grid(row['destination_lat'], row['destination_lon'])
                if dest_grid >= 0:
                    results['st_act_events'].append((time_bin, row['destination_category']))
                    results['st_loc_events'].append((time_bin, dest_grid))
                    results['activity_dist'][row['destination_category']] += 1
                
                # Calculate movement distance
                dist = geodistance(row['origin_lon'], row['origin_lat'], 
                                  row['destination_lon'], row['destination_lat'])
                results['distances'].append(dist)
                
                # Calculate stay duration
                if prev_dt:
                    duration = (dt - prev_dt).total_seconds() / 60  # minutes
                    results['durations'].append(duration)
                
                prev_dt = dt
        
        return results
    
    def calculate_jsd_metrics(self, real_results, gen_results):
        """Calculate JSD metrics (considering multi-day data processing)"""
        metrics = {}
        
        # 1. Movement Distance Distribution (SD)
        max_dist = max(max(real_results['distances']), max(gen_results['distances'])) + 1
        real_dist_hist, _ = np.histogram(real_results['distances'], bins=100, range=(0, max_dist))
        gen_dist_hist, _ = np.histogram(gen_results['distances'], bins=100, range=(0, max_dist))
        metrics['SD'] = js_divergence(real_dist_hist, gen_dist_hist)
        
        # 2. Stay Duration Distribution (SI)
        max_duration = max(max(real_results['durations']), max(gen_results['durations'])) + 1
        real_dur_hist, _ = np.histogram(real_results['durations'], bins=100, range=(0, max_duration))
        gen_dur_hist, _ = np.histogram(gen_results['durations'], bins=100, range=(0, max_duration))
        metrics['SI'] = js_divergence(real_dur_hist, gen_dur_hist)
        
        # 3. Spatiotemporal Activity Distribution (DARD) - Using real data's daily activity distribution as baseline
        # Combine all activity categories
        all_activities = set()
        for day_activities in real_results['daily_activity_dist'].values():
            all_activities.update(day_activities.keys())
        all_activities.update(gen_results['activity_dist'].keys())
        
        # Create activity ID mapping
        activity_map = {act: idx for idx, act in enumerate(all_activities)}
        
        # Calculate real data's daily activity distribution (average)
        real_act_hist = np.zeros(len(activity_map))
        for day_dist in real_results['daily_activity_dist'].values():
            day_hist = np.zeros(len(activity_map))
            for act, count in day_dist.items():
                if act in activity_map:
                    day_hist[activity_map[act]] = count
            real_act_hist += day_hist / sum(day_dist.values())  # Normalize before adding
        real_act_hist /= len(real_results['daily_activity_dist'])
        
        # Calculate generated data's activity distribution
        gen_act_hist = np.zeros(len(activity_map))
        for act, count in gen_results['activity_dist'].items():
            if act in activity_map:
                gen_act_hist[activity_map[act]] = count
        gen_act_hist /= gen_act_hist.sum()  # Normalize
        
        metrics['DARD'] = js_divergence(real_act_hist, gen_act_hist)
        
        # 4. Spatiotemporal Location Distribution (STVD) - Using real data's daily location distribution as baseline
        # Calculate real data's daily location distribution (average)
        real_loc_hist = np.zeros(self.grid_total)
        for day_events in real_results['st_loc_events'].values():
            day_hist = np.zeros(self.grid_total)
            for _, grid_idx in day_events:
                if grid_idx < self.grid_total:
                    day_hist[grid_idx] += 1
            if day_hist.sum() > 0:
                day_hist /= day_hist.sum()  # Normalize
                real_loc_hist += day_hist
        real_loc_hist /= len(real_results['st_loc_events'])
        
        # Calculate generated data's location distribution
        gen_loc_hist = np.zeros(self.grid_total)
        for _, grid_idx in gen_results['st_loc_events']:
            if grid_idx < self.grid_total:
                gen_loc_hist[grid_idx] += 1
        gen_loc_hist /= gen_loc_hist.sum()  # Normalize
        
        metrics['STVD'] = js_divergence(real_loc_hist, gen_loc_hist)
        
        return metrics

# Main function
def main(real_traj_path, gen_traj_path):
    # Load data
    real_df = pd.read_csv(real_traj_path)
    gen_df = pd.read_csv(gen_traj_path)
    
    # Initialize analyzer
    analyzer = TrajectoryAnalyzer(
        grid_size=0.01,
        lat_range=(40.5, 40.9),
        lon_range=(-74.25, -73.7)
    )
    
    # Analyze real trajectories (multi-day)
    real_results = analyzer.analyze_real_trajectories(real_df)
    print(f"Analyzing real trajectories: {len(real_results['durations'])} stay durations, {len(real_results['distances'])} movement distances")
    
    # Analyze generated trajectories (single-day)
    gen_results = analyzer.analyze_generated_trajectories(gen_df)
    print(f"Analyzing generated trajectories: {len(gen_results['durations'])} stay durations, {len(gen_results['distances'])} movement distances")
    
    # Calculate JSD metrics
    metrics = analyzer.calculate_jsd_metrics(real_results, gen_results)
    
    # Print results
    print("\nTrajectory Consistency Analysis Results (Jensen-Shannon Divergence):")
    print(f"1. Movement Distance Distribution (SD): {metrics['SD']:.4f}")
    print(f"2. Stay Duration Distribution (SI): {metrics['SI']:.4f}")
    print(f"3. Activity Distribution (DARD): {metrics['DARD']:.4f}")
    print(f"4. Spatial Location Distribution (STVD): {metrics['STVD']:.4f}")
    
    return metrics

if __name__ == "__main__":
    # File path configuration
    real_traj_path = r'D:\A_Research\A_doing_research\20250526_LLM_causal_inference\dataset\dataset_TSMC2014_NYC.csv'
    gen_traj_path = './output/generated_trajectories.csv'
    
    # Execute analysis
    main(real_traj_path, gen_traj_path)